# 客户购买预测
https://www.kesci.com/home/competition/5c234c6626ba91002bfdfdd3/leaderboard

In [42]:
# 从sklearn 调入所需要的包
# from sklearn import datasets
from sklearn.model_selection import train_test_split #数据分隔出训练集和验证集
import lightgbm as lgb
import numpy as np
import pandas as pd
#导入精度和召回
from sklearn.metrics import precision_score, recall_score
import lightgbm as lgb

In [43]:
path='./'
import os
import json
import gc
# os.system('pip install numba')
from numba import jit
#tqdm
# os.system('pip install tqdm')
from tqdm import tqdm_notebook
from tqdm import tqdm

#Integrated model
# os.system('pip install lightgbm')
import lightgbm as lgb
# os.system('pip install catboost==0.15.2')
import catboost as cbt
# os.system('pip install xgboost')
# import xgboost as xgb

#base import 
import numpy as np
import pandas as pd

# about sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler as std
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import f1_score
#about time
import time
import datetime 
from datetime import datetime, timedelta

#Garbage collection
import gc
# scipy
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
import scipy.spatial.distance as dist
#other
from collections import Counter 
from statistics import mode 
    #warning
import warnings
warnings.filterwarnings("ignore")
import json 
import math
from itertools import product
import ast 

In [44]:
path='./'
train = pd.read_csv(path+'input/train_set.csv')
test = pd.read_csv(path+'input/test_set.csv')
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25317 entries, 0 to 25316
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         25317 non-null  int64 
 1   age        25317 non-null  int64 
 2   job        25317 non-null  object
 3   marital    25317 non-null  object
 4   education  25317 non-null  object
 5   default    25317 non-null  object
 6   balance    25317 non-null  int64 
 7   housing    25317 non-null  object
 8   loan       25317 non-null  object
 9   contact    25317 non-null  object
 10  day        25317 non-null  int64 
 11  month      25317 non-null  object
 12  duration   25317 non-null  int64 
 13  campaign   25317 non-null  int64 
 14  pdays      25317 non-null  int64 
 15  previous   25317 non-null  int64 
 16  poutcome   25317 non-null  object
 17  y          25317 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 3.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10852 

In [45]:
train.describe()

,ID,age,balance,day,duration,campaign,pdays,previous,y
count,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000
mean,12659.000000,40.935379,1357.555082,15.835289,257.732393,2.772050,40.248766,0.591737,0.116957
std,7308.532719,10.634289,2999.822811,8.319480,256.975151,3.136097,100.213541,2.568313,0.321375
min,1.000000,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,0.000000
25%,6330.000000,33.000000,73.000000,8.000000,103.000000,1.000000,-1.000000,0.000000,0.000000
50%,12659.000000,39.000000,448.000000,16.000000,181.000000,2.000000,-1.000000,0.000000,0.000000
75%,18988.000000,48.000000,1435.000000,21.000000,317.000000,3.000000,-1.000000,0.000000,0.000000
max,25317.000000,95.000000,102127.000000,31.000000,3881.000000,55.000000,854.000000,275.000000,1.000000


In [46]:
train.corr()

,ID,age,balance,day,duration,campaign,pdays,previous,y
ID,1.000000,0.008465,0.032719,-0.020171,0.229149,-0.038265,0.065807,0.047028,0.556627
age,0.008465,1.000000,0.093740,-0.016070,0.000416,0.006171,-0.026431,0.006575,0.029916
balance,0.032719,0.093740,1.000000,0.010245,0.026042,-0.010419,0.001032,0.015792,0.057564
day,-0.020171,-0.016070,0.010245,1.000000,-0.031946,0.168830,-0.092892,-0.050706,-0.031886
duration,0.229149,0.000416,0.026042,-0.031946,1.000000,-0.087780,0.000040,0.001315,0.394746
campaign,-0.038265,0.006171,-0.010419,0.168830,-0.087780,1.000000,-0.089224,-0.031667,-0.075173
pdays,0.065807,-0.026431,0.001032,-0.092892,0.000040,-0.089224,1.000000,0.411688,0.107565
previous,0.047028,0.006575,0.015792,-0.050706,0.001315,-0.031667,0.411688,1.000000,0.088337
y,0.556627,0.029916,0.057564,-0.031886,0.394746,-0.075173,0.107565,0.088337,1.000000


In [47]:
train.job.value_counts()

blue-collar      5456
management       5296
technician       4241
admin.           2909
services         2342
retired          1273
self-employed     884
entrepreneur      856
unemployed        701
housemaid         663
student           533
unknown           163
Name: job, dtype: int64

## 串接训练集和测试集，切记做lable encoder前要串接数据

In [48]:
test['y']=-1
data = train.append(test).reset_index(drop=True)
# data = pd.concat([train,test]).reset_index(drop=True)
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


## label encoder

In [49]:
# cat_col = [i for i in data.select_dtypes(object).columns if i not in ['ID','y']]
# for i in tqdm_notebook(cat_col):
#     lbl = LabelEncoder()
# #     data['count_' + i] = data.groupby([i])[i].transform('count')
#     data[i] = lbl.fit_transform(data[i].astype(str))


In [50]:
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


### 特征工程

In [51]:
sparse_feature= ['campaign','contact','default','education','housing','job','loan','marital','month','poutcome']
dense_feature=['age', 'balance','day','duration','campaign','pdays','previous']
for d in tqdm(sparse_feature):
    aggs={}
    for s in sparse_feature:
        temp=data.groupby(d)[s].agg(['count','nunique']).reset_index().rename(columns={'count':'count_'+str(d)+'_'+str(s),'nunique':'nunique_'+str(d)+'_'+str(s)})
        data=pd.merge(data,temp,on=d,how='left')

for d in tqdm(sparse_feature):
    aggs={}
    for s in dense_feature:
        temp=data.groupby(d)[s].agg(['mean','max','min','std']).reset_index().rename(columns={'mean':str(d)+'_mean_'+str(s),'max':str(d)+'_max_'+str(s),'min':str(d)+'_min_'+str(s),'std':str(d)+'_std_'+str(s)})
        data=pd.merge(data,temp,on=d,how='left')

100%|██████████| 10/10 [03:09<00:00, 18.92s/it]


## lightgbm支持类别特征，因此可以不需要做lable encoder

In [52]:
cat_col = [i for i in data.select_dtypes(object).columns if i not in ['ID','y']]
    
for i in cat_col:
    data[i] = data[i].astype('category')

In [25]:
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36169 entries, 0 to 36168
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   ID         36169 non-null  int64
 1   age        36169 non-null  int64
 2   job        36169 non-null  int32
 3   marital    36169 non-null  int32
 4   education  36169 non-null  int32
 5   default    36169 non-null  int32
 6   balance    36169 non-null  int64
 7   housing    36169 non-null  int32
 8   loan       36169 non-null  int32
 9   contact    36169 non-null  int32
 10  day        36169 non-null  int64
 11  month      36169 non-null  int32
 12  duration   36169 non-null  int64
 13  campaign   36169 non-null  int64
 14  pdays      36169 non-null  int64
 15  previous   36169 non-null  int64
 16  poutcome   36169 non-null  int32
 17  y          36169 non-null  int64
dtypes: int32(9), int64(9)
memory usage: 3.7 MB


1ID Int客户唯一标识

2age Int客户年龄

3job String客户的职业

4marital String婚姻状况

5education String受教育水平

6default String是否有违约记录

7balance Int每年账户的平均余额

8housing String是否有住房贷款

9loan String是否有个人贷款

10contact String与客户联系的沟通方式

11day Int最后一次联系的时间（几号）

12month String最后一次联系的时间（月份）

13duration Int最后一次联系的交流时长

14campaign Int在本次活动中，与该客户交流过的次数

15pdays Int距离上次活动最后一次联系该客户，过去了多久（999表示没有联系过）16previousInt在本次活动之前，与该客户交流过的次数

17poutcome String上一次活动的结果

18y Int预测客户是否会订购定期存款业务

In [53]:
feats = [i for i in data.columns if i not in ['ID','y']]
feats

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'count_campaign_campaign',
 'nunique_campaign_campaign',
 'count_campaign_contact',
 'nunique_campaign_contact',
 'count_campaign_default',
 'nunique_campaign_default',
 'count_campaign_education',
 'nunique_campaign_education',
 'count_campaign_housing',
 'nunique_campaign_housing',
 'count_campaign_job',
 'nunique_campaign_job',
 'count_campaign_loan',
 'nunique_campaign_loan',
 'count_campaign_marital',
 'nunique_campaign_marital',
 'count_campaign_month',
 'nunique_campaign_month',
 'count_campaign_poutcome',
 'nunique_campaign_poutcome',
 'count_contact_campaign',
 'nunique_contact_campaign',
 'count_contact_contact',
 'nunique_contact_contact',
 'count_contact_default',
 'nunique_contact_default',
 'count_contact_education',
 'nunique_contact_education',
 'count_contact_housing',
 'nunique_contact_housi

In [27]:
import lightgbm as lgb
model = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=30, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary',metric= 'auc',
    subsample=0.95, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.02, random_state=2017
    )
data1 =data[data['y']!=-1][feats]
label1 =data[data['y']!=-1]['y']
testx= data[data['y']==-1][feats]

train_x, test_x, train_y, test_y = train_test_split(data1, label1, test_size=0.3, random_state=42)
model.fit(train_x,train_y)
test_pre = model.predict_proba(testx)[:,1]#, num_iteration=model.best_iteration

In [64]:
pre=data[data['y']==-1][['ID']]
pre['pred']=test_pre
pre.to_csv('baseline2.csv',index=False )
print(len(pre))

10852


## 使用kfolder进行集成

In [54]:
from sklearn.model_selection import KFold,StratifiedKFold
n_splits=5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
for train_idx, val_idx in kfold.split(train_x):
    print(train_idx)
    break

[    0     1     2 ... 25314 25315 25316]


In [55]:
#5折交叉验证
from sklearn.model_selection import KFold,StratifiedKFold
n_splits=5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)


train_x = data[data['y']!=-1][feats]
train_y = data[data['y']!=-1]['y']


res=data[data['y']==-1][['ID']]
res['pred'] = 0


test_x= data[data['y']==-1][feats]

for train_idx, val_idx in tqdm_notebook(kfold.split(train_x)):
    model.random_state = model.random_state + 1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    valid_x1 = train_x.loc[val_idx]
    valid_y1 = train_y.loc[val_idx]
    #注意此处可以显式指定 categorical_feature特征
    model.fit(train_x1, train_y1,eval_set=[(train_x1, train_y1),(valid_x1, valid_y1)],categorical_feature=cat_col,eval_metric='auc',early_stopping_rounds=100)
    res['pred'] += model.predict_proba(test_x)[:,1]

res['pred'] = res['pred']/n_splits
res.to_csv('./submission5_fold.csv', index=False)

id_1's auc: 0.929578
[260]	training's auc: 0.964669	valid_1's auc: 0.929622
[261]	training's auc: 0.964741	valid_1's auc: 0.929617
[262]	training's auc: 0.964824	valid_1's auc: 0.929652
[263]	training's auc: 0.964899	valid_1's auc: 0.92967
[264]	training's auc: 0.964975	valid_1's auc: 0.929676
[265]	training's auc: 0.965026	valid_1's auc: 0.929653
[266]	training's auc: 0.965099	valid_1's auc: 0.929677
[267]	training's auc: 0.965163	valid_1's auc: 0.929672
[268]	training's auc: 0.965242	valid_1's auc: 0.929698
[269]	training's auc: 0.965322	valid_1's auc: 0.929728
[270]	training's auc: 0.965399	valid_1's auc: 0.929701
[271]	training's auc: 0.965472	valid_1's auc: 0.929661
[272]	training's auc: 0.965558	valid_1's auc: 0.929645
[273]	training's auc: 0.96563	valid_1's auc: 0.929655
[274]	training's auc: 0.965725	valid_1's auc: 0.929764
[275]	training's auc: 0.965783	valid_1's auc: 0.929756
[276]	training's auc: 0.965845	valid_1's auc: 0.929731
[277]	training's auc: 0.965943	valid_1's auc: 